In [9]:
#!/usr/bin/env python
import RPi.GPIO as GPIO
import time
import subprocess

TRIG = 11
ECHO = 12
pid = False  # Store the sound process
MIN_V = 45
MAX_V = 80
MIN_T = 0.00011764705
MAX_T = 0.02352941176

def dur_to_vol(t):
    return MAX_V - ((MAX_V - MIN_V) / (MAX_T - MIN_T))*t

def sonic_sound_setup():
    global pid
    GPIO.setmode(GPIO.BOARD)
    GPIO.setup(TRIG, GPIO.OUT)
    GPIO.setup(ECHO, GPIO.IN)
    pid = sound_proc(440)
    set_volume(MIN_V)

def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

def duration():
    GPIO.output(TRIG, 0)
    time.sleep(0.000002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)
    while GPIO.input(ECHO) == 0: pass
    time1 = time.time()
    while GPIO.input(ECHO) == 1: pass
    time2 = time.time()
    t = time2 - time1
    if (MIN_T < t < MAX_T): return t 
    else: return False

def set_volume(percent):
    subprocess.call(['amixer', 'set', 'PCM', str.format('{}%', percent)])

if __name__ == "__main__":
    try:
        sonic_sound_setup()
        while True:
            t = duration()
            if t:
                v = dur_to_vol(t)
                set_volume(v)
            else:
                pass
            time.sleep(0.1)  # Gotta sleep to stop this from being pre-empted
            
    except KeyboardInterrupt:
        print('User interrupted')
    finally:
        GPIO.cleanup()
        pid.kill()

User interrupted
